In [41]:
#!pip install checklist

In [42]:
#!pip install simpletransformers

In [6]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score

In [7]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
train_data = pd.read_csv('/content/data/olid-train.csv')

data = data[['text','labels']]
train_data = train_data[['text','labels']]
data.head(3)

,text,labels
0,@USER @USER Who the hell does he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1


In [8]:
nlp = spacy.load('en_core_web_sm')

In [9]:
perturbed_data = list(nlp.pipe(data['text']))

In [10]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [11]:
np.random.seed(42)
ret = Perturb.perturb(data['text'], Perturb.add_typos, typos = 2)
ret.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring kille dantifa arti... URL via @USER URL']

In [12]:
typo_data = []
for i in range(len(ret.data)):
  typo_data.append(ret.data[i][1]) # tweet with typos added

typo_data[:3]

['@USER @USER Who the hell doe she thinkh e is?',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring kille dantifa arti... URL via @USER URL',
 '#OrrinHatch I can’t eblieve this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewde 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!']

In [13]:
data_typo = pd.DataFrame({'text':typo_data,'label':data['labels']})
data_typo.head()

,text,label
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [14]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [15]:
model.train_model(train_data)

  0%|          | 0/13240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(1655, 0.49876208406200584)

In [16]:
predictions, raw_outputs = model.predict(list(data_typo['text']))
pred, out = model.predict(list(data['text']))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [17]:
predicted_typo_labels = pd.DataFrame({'text':data_typo.text, 'labels': predictions})
predicted_typo_labels.head()

,text,labels
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,0
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [18]:
result, model_outputs, wrong_predictions = model.eval_model(data_typo, report_typo = classification_report)
result_d, model_outputs_d, wrong_predictions_d = model.eval_model(data, report_normaldata = classification_report)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [19]:
print(result['report_typo'])
print("\n")
print(result_d['report_normaldata'])

              precision    recall  f1-score   support

         0.0       0.72      0.94      0.82        50
         1.0       0.91      0.64      0.75        50

    accuracy                           0.79       100
   macro avg       0.82      0.79      0.79       100
weighted avg       0.82      0.79      0.79       100



              precision    recall  f1-score   support

         0.0       0.75      0.96      0.84        50
         1.0       0.94      0.68      0.79        50

    accuracy                           0.82       100
   macro avg       0.85      0.82      0.82       100
weighted avg       0.85      0.82      0.82       100



In [20]:
print(model_outputs[:5])

[[-0.42236328  0.72753906]
 [ 0.50537109 -0.65332031]
 [-1.02929688  1.11328125]
 [ 0.25952148 -0.52294922]
 [-0.95849609  1.08886719]]


In [21]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[47  3]
 [18 32]]


### Provide 3 examples when the model failed to assign the correct label after perturbation.

In [22]:
print(data['labels'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [23]:
print(predicted_typo_labels['labels'][:10])

0    1
1    0
2    1
3    0
4    1
5    1
6    0
7    1
8    0
9    1
Name: labels, dtype: int64


In [24]:
failed_texts=[]
failed_labels=[]
for i in range(len(data)):
  if predicted_typo_labels['labels'][i] != data['labels'][i]:
    failed_texts.append(predicted_typo_labels['text'][i])
    failed_labels.append(predicted_typo_labels['labels'][i])

In [25]:
failed_examples = pd.DataFrame({'text':failed_texts, 'labels': failed_labels})
failed_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails fly afte...,0
1,@USER @USER I'll use that one the next time im...,0
2,#Christian #America – If we go by #Trump’s exa...,0
3,#Democrats #Liberals you are being #threatened...,0
4,#Antifa: Take note ofh ow to protest with civi...,0
5,@USER oh ufck off 😂😂,0
6,"#Emmys Well, omst of those ex-crackheads are ...",0
7,#BoycottNike campaig nmust continue and grow ...,0
8,#ANTIFA are bedfellows with The Democratic Soc...,0
9,@USER Anitfa ahs TS level influence. It's scary.,0


# 6. Negation *(4.5 points)* 

In [26]:
perturbed_data = list(nlp.pipe(data['text']))

In [28]:
ret = Perturb.perturb(perturbed_data, Perturb.add_negation) # negated data
ret.data[:5]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [29]:
negated_text = []
for i in range(len(ret.data)):
  negated_text.append(ret.data[i][1])

negated_text[:5]

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba"]

In [30]:
neg_predictions, neg_raw_outputs = model.predict(negated_text)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [31]:
predicted_neg_labels = pd.DataFrame({'text':negated_text, 'labels': data['labels']})
predicted_neg_labels.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [32]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_neg_labels, neg_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [33]:
print(result['neg_report'])

              precision    recall  f1-score   support

         0.0       0.73      0.96      0.83        50
         1.0       0.94      0.64      0.76        50

    accuracy                           0.80       100
   macro avg       0.83      0.80      0.79       100
weighted avg       0.83      0.80      0.79       100



In [34]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], predictions))

Confusion Matrix : 
[[47  3]
 [18 32]]


### Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion?

In [35]:
predicted_neg_labels['text'][:10]

0     @USER @USER Who the hell doesn't he think he is?
1    #BREAKING. #Greece: Molotov cocktails don't fl...
2    #OrrinHatch I can’t believe this sexist , clue...
3    @USER @USER I'll use that one the next time im...
4    0-1 didn't lose my acca on the first fucking f...
5    #Bakersfield is not why we need gun control! S...
6    #Christian #America – If we go by #Trump’s exa...
7    @USER @USER @USER She is not the most disingen...
8    #Democrats #Liberals you are being #threatened...
9    699. Just didn't want to tell you you should H...
Name: text, dtype: object

In [36]:
predicted_neg_labels_df = pd.DataFrame({'text':negated_text, 'labels': neg_predictions})
predicted_neg_labels_df.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 didn't lose my acca on the first fucking f...,1


In [37]:
failed_neg_texts=[]
failed_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] != data['labels'][i]:
    failed_neg_texts.append(predicted_neg_labels['text'][i])
    failed_neg_labels.append(predicted_neg_labels['labels'][i])

In [38]:
failed_neg_examples = pd.DataFrame({'text':failed_neg_texts, 'labels': failed_neg_labels})
failed_neg_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails don't fl...,1
1,@USER @USER I'll use that one the next time im...,1
2,#Christian #America – If we go by #Trump’s exa...,1
3,#Democrats #Liberals you are being #threatened...,1
4,#Antifa: don't take note of how to protest wit...,1
5,"#Emmys Well, most of those ex-crackheads are ...",1
6,#Nigeria #Naija #9ja 'You are not the most inc...,1
7,#BoycottNike campaign must not continue and gr...,1
8,#ANTIFA are not bedfellows with The Democratic...,1
9,@USER Antifa doesn't have TS level influence. ...,1


In [39]:
success_neg_texts=[]
success_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] == data['labels'][i]:
    success_neg_texts.append(predicted_neg_labels['text'][i])
    success_neg_labels.append(predicted_neg_labels['labels'][i])

In [40]:
successful_neg_examples = pd.DataFrame({'text':success_neg_texts, 'labels': success_neg_labels})
successful_neg_examples

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,"#OrrinHatch I can’t believe this sexist , clue...",1
2,0-1 didn't lose my acca on the first fucking f...,1
3,#Bakersfield is not why we need gun control! S...,1
4,@USER @USER @USER She is not the most disingen...,1
...,...,...
75,@USER He is obviously getting not suspended. H...,0
76,#Canada - EXCLUSIVE: #Trudeau #Liberals don't ...,0
77,@USER @USER ...than why didn't you show us how...,0
78,@USER @USER @USER You don't have yet to answer...,0


## 7. Creating negated examples